In [ ]:
clear

In [12]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /Users/twei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from transformers import BertTokenizer, BertModel
import torch
huggingface_model = 'bert-base-uncased'
model = BertModel.from_pretrained(huggingface_model)
tokenizer = BertTokenizer.from_pretrained(huggingface_model)


2023-02-06 11:06:34.548512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expe

In [14]:
df_dbpedia = pd.read_csv("./files/dbpedia_classes.csv")
df_imageNet = pd.read_csv("./files/ImageNet.csv")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.replace(", ",",")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.split(",")

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')


In [16]:
dbp=model.encode(['zoo'])
imn = model.encode(['goldfish'])
cosine_similarity(dbp,imn)

array([[0.30338714]], dtype=float32)

### Sentence Embeddings using Siamese BERT-Networks
BERT (Devlin et al., 2018) and RoBERTa (Liu et al., 2019) has set a new state-of-the-art performance on sentence-pair regression tasks like semantic textual similarity (STS). However, it requires that both sentences are fed into the network, which causes a massive computational overhead: Finding the most similar pair in a collection of 10,000 sentences requires about 50 million inference computations (~65 hours) with BERT. 

**Sentence-BERT (SBERT)**, a modification of the pretrained BERT network that use siamese and triplet network structures to derive semantically meaningful sentence embeddings that can be compared using cosine-similarity. This reduces the effort for finding the most similar pair from 65 hours with BERT / RoBERTa to about 5 seconds with SBERT, while maintaining the accuracy from BERT.

In [113]:
len(embeddings[0])

768

In [17]:
#Our sentences we like to encode

sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.04094855e-01  5.27476907e-01  1.17977226e+00  1.94366127e-01
  1.59113780e-01  5.45549631e-01  2.81810194e-01  1.21591777e-01
  2.91968822e-01 -3.36857527e-01 -2.49655992e-01  2.93203056e-01
  1.43483192e-01  3.80201668e-01 -2.52250791e-01 -2.62337804e-01
 -2.69969016e-01  4.68081385e-02  3.57254535e-01 -3.58307898e-01
 -2.89185271e-02  6.85313568e-02 -7.84855545e-01 -2.61793017e-01
  1.02695477e+00 -4.05540109e-01  1.93897337e-01 -1.65970296e-01
 -2.89741755e-01 -1.07935898e-01 -6.31144822e-01 -4.68853146e-01
  7.16446936e-01 -9.49059427e-01 -2.94918679e-02  1.37850773e-02
  7.03230575e-02 -2.72184908e-02 -6.25346750e-02 -5.84831536e-01
 -9.02268052e-01 -9.56011832e-01  4.95161563e-01  1.95099754e-04
 -1.52250671e+00 -6.86906755e-01 -9.08192575e-01 -1.79839581e-01
 -6.49179816e-01 -2.96637136e-02 -1.83349729e+00  1.43428952e-01
  1.93934143e-01  1.91236306e-02 -4.56058532e-01  9.44655776e-01
  1.66153

In [33]:
def get_max_cos_ratio_10():
    max_ratio_list = []
    max_label_list = []
    for class_list in df_imageNet_10['Class Name']:
      max_val = 0
      max_label = ''
      for item in class_list:
        sentence_embeddings_imageNet = model.encode([item])
        for label in df_dbpedia['label']:
            val = cosine_similarity(sentence_embeddings_imageNet,model.encode([label]))
            if max_val< val:
                max_val = val
                max_label = label
      max_ratio_list.append(max_val)
      max_label_list.append(max_label)
    df_imageNet_10['max_similarity_ratio'] = max_ratio_list
    df_imageNet_10['dbpedia_class'] = max_label_list
    df_imageNet_10.rename(columns = {"Class Name":"imageNet_class"}, inplace=True)
    
            
get_max_cos_ratio_10()

/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1651988033.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imageNet_10['max_similarity_ratio'] = max_ratio_list
/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1651988033.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imageNet_10['dbpedia_class'] = max_label_list
/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1651988033.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [ ]:
for i in range(0,1000):
    for el_db in db_list:
        a = db_list.index(el_db) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
                if e in model.key_to_index:
                    m_db[x] = model[e]
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            
            sim = get_cosine_similarity(el_img_vec[i],dbpedia)
            dataframe.iat[a,i] = sim            
        else:
            try:
                sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe.iat[a,i] = sim                
            except:
                dataframe.iat[a,i] = "Not Present"  # this happens if the dbpedia word is not present in the model


In [45]:
word_imageNet = 'Tinca tinca'
word_dbpedia  = 'fish' 
emb_im = model.encode([word_imageNet])
emb_db = model.encode([word_dbpedia])
print("Sentence:", word_imageNet)
print("Embedding:", emb_im)
print("")

Sentence: Tinca tinca
Embedding: [[-1.77918021e-02  2.84497082e-01  1.25613534e+00  1.84398055e-01
   5.27764559e-01  3.37899953e-01 -7.49733865e-01  9.83704388e-01
   5.05480282e-02  2.51939204e-02 -4.99189287e-01  6.97399139e-01
   5.66759229e-01  5.93078256e-01  2.40798429e-01  3.51313442e-01
  -5.66609323e-01 -3.16470377e-02  3.90807897e-01 -7.21403837e-01
  -5.83431870e-02  6.87481225e-01 -2.69172579e-01 -1.04580200e+00
  -3.55123073e-01 -4.69993591e-01  2.98205223e-02 -1.66961944e+00
  -2.97227114e-01  1.01135699e-02 -5.59428513e-01 -2.56886631e-01
   4.27166633e-02 -3.54395628e-01 -5.45405686e-01  5.36320023e-02
  -3.09007943e-01  8.54490697e-03  6.38954118e-02 -7.13969767e-01
   1.86973321e+00 -4.34361577e-01  6.77858889e-01  1.14543319e-01
  -1.05179036e+00  4.82837446e-02 -1.16952442e-01  4.18478437e-02
  -3.46781045e-01 -1.63570154e+00 -7.37211883e-01 -9.60573435e-01
   7.71406949e-01  2.24669218e-01  9.16563496e-02 -1.01228051e-01
   3.33636194e-01 -2.40858495e-01  4.228223

In [48]:
df_imageNet_100 = df_imageNet.iloc[:100]

In [49]:
def get_max_cos_ratio():
    max_ratio_list = []
    max_label_list = []
    for class_list in df_imageNet_100['Class Name']:
      max_val = 0
      max_label = ''
      for item in class_list:
        sentence_embeddings_imageNet = model.encode([item])
        for label in df_dbpedia['label']:
            val = cosine_similarity(sentence_embeddings_imageNet,model.encode([label]))
            if max_val< val:
                max_val = val
                max_label = label
      max_ratio_list.append(max_val)
      max_label_list.append(max_label)
    df_imageNet_100['max_similarity_ratio'] = max_ratio_list
    df_imageNet_100['dbpedia_class'] = max_label_list
    df_imageNet_100.rename(columns = {"Class Name":"imageNet_class"}, inplace=True)
    
            
get_max_cos_ratio()

/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1001000062.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imageNet_100['max_similarity_ratio'] = max_ratio_list
/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1001000062.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imageNet_100['dbpedia_class'] = max_label_list
/var/folders/rl/bmzv82250753s6nrlbm67tm40000gn/T/ipykernel_24467/1001000062.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [51]:
df_imageNet_100.head(30)

,Class ID,imageNet_class,max_similarity_ratio,dbpedia_class
0,0,"[tench, Tinca tinca]",[[0.79340786]],type
1,1,"[goldfish, Carassius auratus]",[[0.70011944]],animanga character
2,2,"[great white shark, white shark, man-eater, ma...",[[0.91215545]],type
3,3,"[tiger shark, Galeocerdo cuvieri]",[[0.709738]],medician
4,4,"[hammerhead, hammerhead shark]",[[0.7021628]],factory
5,5,"[electric ray, crampfish, numbfish, torpedo]",[[0.81626856]],fish
6,6,[stingray],[[0.7539027]],fish
7,7,[cock],[[0.7068688]],Hormone
8,8,[hen],[[0.6958789]],Manor
9,9,"[ostrich, Struthio camelus]",[[0.70469856]],animanga character


In [5]:
img_net = pd.read_csv("./files/ImageNet.csv")
img_net_list = img_net["Class Name"].tolist()

dbpedia = pd.read_csv("./files/dbpedia_classes.csv")
db_list = dbpedia["label"].tolist()

In [43]:
dataframe = pd.DataFrame(index =db_list ,columns =img_net_list)
dataframe=dataframe.fillna(0)

In [26]:
df_imageNet_10['Class Name']

0                                 [tench, Tinca tinca]
1                        [goldfish, Carassius auratus]
2    [great white shark, white shark, man-eater, ma...
3                    [tiger shark, Galeocerdo cuvieri]
4                       [hammerhead, hammerhead shark]
5         [electric ray, crampfish, numbfish, torpedo]
6                                           [stingray]
7                                               [cock]
8                                                [hen]
9                          [ostrich, Struthio camelus]
Name: Class Name, dtype: object

In [30]:
df_imageNet_2 = df_imageNet.iloc[:2]

In [31]:
df_imageNet_2

,Class ID,Class Name
0,0,"[tench, Tinca tinca]"
1,1,"[goldfish, Carassius auratus]"


In [53]:
df_imageNet['Class Name'][:99]

0                                  [tench, Tinca tinca]
1                         [goldfish, Carassius auratus]
2     [great white shark, white shark, man-eater, ma...
3                     [tiger shark, Galeocerdo cuvieri]
4                        [hammerhead, hammerhead shark]
                            ...                        
94                                        [hummingbird]
95                                            [jacamar]
96                                             [toucan]
97                                              [drake]
98            [red-breasted merganser, Mergus serrator]
Name: Class Name, Length: 99, dtype: object

In [96]:
def get_sim_ratio(df: pd.DataFrame):
    df_cp = df
    max_ratio_list = []
    max_label_list = []
    j = 650
    for class_list in df_imageNet['Class Name'][650:1]:
        for item in class_list:
            sentence_embeddings_imageNet = model.encode([item])
            i = 0
            for label in df_dbpedia['label']:
                val =  cosine_similarity(sentence_embeddings_imageNet,model.encode([label]))[0][0]
                #print('value is:' + str(val) + 'value in df' + str(df_cp.iat[i,j]))
                if  val > df_cp.iat[i,j]:
                    df_cp.iat[i,j] = val
                i+=1
        j+=1
        
    return df_cp
    
df_filled = get_sim_ratio(df_filled)

In [99]:
df_filled_sorted = df_filled.apply(lambda s: s.abs().nlargest(3).index.tolist(), axis=0)

In [100]:
df_filled_sorted.replace(" ","", regex=True)

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
0,type,animangacharacter,type,medician,factory,fish,fish,Hormone,Manor,animangacharacter,...,horse,fungus,type,Gnetophytes,fungus,planet,animangacharacter,cape,Resume,company
1,name,manhua,name,regency,work,powerstation,sea,attack,name,bird,...,type,mollusca,Reference,engine,murderer,archaea,polysaccharide,settlement,reign,activity
2,department,Cardinaldirection,image,animangacharacter,bone,scientist,reptile,man,type,mammal,...,department,bacteria,place,GlobularSwarm,crater,galaxy,cape,Blazon,Reference,name


In [108]:
df_filled_sorted = df_filled_sorted.replace(" ","", regex=True)

In [109]:
df_filled_sorted

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
0,type,animangacharacter,type,medician,factory,fish,fish,Hormone,Manor,animangacharacter,...,horse,fungus,type,Gnetophytes,fungus,planet,animangacharacter,cape,Resume,company
1,name,manhua,name,regency,work,powerstation,sea,attack,name,bird,...,type,mollusca,Reference,engine,murderer,archaea,polysaccharide,settlement,reign,activity
2,department,Cardinaldirection,image,animangacharacter,bone,scientist,reptile,man,type,mammal,...,department,bacteria,place,GlobularSwarm,crater,galaxy,cape,Blazon,Reference,name


In [111]:
df_filled_sorted.to_csv('mapping_results_bert.csv')

In [104]:
df_filled_sorted

,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
0,type,animanga character,type,medician,factory,fish,fish,Hormone,Manor,animanga character,...,horse,fungus,type,Gnetophytes,fungus,planet,animanga character,cape,Resume,company
1,name,manhua,name,regency,work,power station,sea,attack,name,bird,...,type,mollusca,Reference,engine,murderer,archaea,polysaccharide,settlement,reign,activity
2,department,Cardinal direction,image,animanga character,bone,scientist,reptile,man,type,mammal,...,department,bacteria,place,Globular Swarm,crater,galaxy,cape,Blazon,Reference,name


In [7]:
#el_img_vec[48][57] = 0.0
zero_vec = ""
for i in range(0,len(el_img_vec)):
    for el_db in db_list:
        a = db_list.index(el_db) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
#                 if e in model.key_to_index:
                    m_db[x] = model[e]
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            
            sim = get_cosine_similarity(el_img_vec[i],dbpedia)
            dataframe.iat[a,i] = sim            
        else:
            try:
                sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe.iat[a,i] = sim                
            except:
                dataframe.iat[a,i] = "Not Present"  # this happens if the dbpedia word is not present in the model


,"tench, Tinca tinca","goldfish, Carassius auratus","great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',","tiger shark, Galeocerdo cuvieri","hammerhead, hammerhead shark","electric ray, crampfish, numbfish, torpedo",stingray,cock,hen,"ostrich, Struthio camelus",...,"buckeye, horse chestnut, conker",coral fungus,agaric,gyromitra,"stinkhorn, carrion fungus",earthstar,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bolete,"ear, spike, capitulum","toilet tissue, toilet paper, bathroom tissue"
company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind motor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Windmill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
woman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
